In [1]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn import preprocessing
import sys
sys.path.insert(0, 'XXXX-XXXX')
from read_db import *

In [ ]:
path = r"XXXX-XXXX"
con = sqlite3.connect(path)
root = pd.read_sql_query("SELECT  * FROM avgPlayerStatistics", con)
pd.set_option('display.max_columns', None)

In [101]:
display(root)

,index,assists,blocks,commited_fouls,commited_penalty,conceded,drawn_fouls,dribble_attempts,dribble_past,dribble_success,fixture_id,interceptions,key_passes,minutes,missed_penalty,number,offside,on_goal_shots,pass_accuracy,player_id,player_position,rating,red,saved_penalty,saves,scored_penalty,timestamp,total_duels,total_goals,total_passes,total_shots,total_tackels,won_duels,won_penalty,yellow
0,0,0.000,0.000,0.000,0.000,1.000,0.400,0.200,0.000,0.200,37823,0.000,0.000,90.000,0.000,1.000,0.000,0.000,49.200,21378,G,6.720,0.000,0.000,2.400,0.000,1552158000,0.800,0.000,13.400,0.000,0.000,0.800,0.000,0.000
1,1,0.000,1.000,0.000,0.000,0.000,1.000,1.000,0.600,0.600,37823,1.400,0.200,72.200,0.000,21.400,0.000,0.200,62.800,21383,D,0.000,0.000,0.000,0.000,0.000,1552158000,9.600,0.000,21.200,0.800,1.800,6.000,0.000,0.200
2,2,0.000,0.800,0.000,0.000,0.000,0.800,0.000,0.400,0.000,37823,0.800,0.000,90.000,0.000,2.000,0.000,0.000,83.400,21380,D,0.000,0.000,0.000,0.000,0.000,1552158000,7.600,0.000,30.200,0.000,0.600,4.400,0.000,0.000
3,3,0.000,0.200,0.000,0.000,0.000,1.200,0.800,0.600,0.400,37823,0.800,0.600,90.000,0.000,4.000,0.000,0.000,77.400,2855,D,0.000,0.000,0.000,0.000,0.000,1552158000,7.000,0.000,20.800,0.200,1.000,3.600,0.000,0.000
4,4,0.000,0.000,0.000,0.000,0.000,0.600,0.200,0.400,0.200,37823,2.000,0.000,84.200,0.000,25.000,0.000,0.000,74.000,21381,D,6.800,0.000,0.000,0.000,0.000,1552158000,7.600,0.000,33.400,0.200,0.200,5.200,0.000,0.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322757,23029,0.200,0.000,0.000,0.000,0.000,1.000,0.000,0.800,0.000,705683,1.200,0.600,62.200,0.000,21.000,0.000,0.000,71.400,15327,M,6.940,0.000,0.000,0.000,0.000,1622397600,5.400,0.000,28.800,0.400,0.200,2.800,0.000,0.000
322758,23030,0.000,0.200,0.000,0.000,0.000,0.800,0.200,0.200,0.000,705683,0.600,0.400,38.000,0.000,8.800,0.000,0.000,70.000,43304,M,0.000,0.000,0.000,0.000,0.000,1622397600,3.800,0.000,14.800,0.000,0.800,1.800,0.000,0.000
322759,23031,0.200,0.000,0.000,0.000,0.000,2.600,2.200,0.600,1.200,705683,0.600,0.400,48.000,0.000,78.000,0.000,0.000,38.800,41930,F,0.000,0.000,0.000,0.000,0.000,1622397600,10.400,0.000,8.400,0.600,0.200,5.600,0.200,0.000
322760,23032,0.000,0.000,0.000,0.000,0.000,1.000,1.000,0.667,0.000,705683,0.667,0.000,44.000,0.000,68.333,0.000,0.333,50.000,41188,F,6.667,0.000,0.000,0.000,0.000,1622397600,8.667,0.333,8.667,1.000,0.333,3.000,0.000,0.667


In [142]:
# root['on_goal_shots'].plot.hist(bins=200)
# root = root.drop("player_position", axis=1)
# root = root.drop("index", axis=1)
root.insert(0, 'fixture_id', root.pop('fixture_id'))
root.insert(1, 'player_id', root.pop('player_id'))
root.insert(2, 'timestamp', root.pop('timestamp'))
# display(root)



In [57]:
#für jede position sum(normalize mal gewichte)
#mit player, fixture_id in db speichern 
#pro fixture => elo und spielerkraft in eine spalte 
#erstelle Dataset

In [143]:
x = root.to_numpy() #returns a numpy array
robust_scaler = preprocessing.RobustScaler()
x_scaled = robust_scaler.fit_transform(x)
all_norm = pd.DataFrame(x_scaled, columns=root.columns)
#display(all_norm)


In [144]:
x = root.to_numpy() #returns a numpy array
x1 = x[:, 3:] # every column from colum 3(timestamp)
robust_scaler = preprocessing.RobustScaler()
x_scaled = robust_scaler.fit_transform(x1)
x_scaled = np.concatenate(( x[:,[0,1,2]], x_scaled), axis=1)
final = pd.DataFrame(x_scaled, columns=root.columns)
columns = root.keys()
for i in range(3): # fixture_id bis timestamp zu int
    final[columns[i]] = pd.to_numeric(final[columns[i]], downcast='integer')
#display(final)


In [141]:
columns = root.keys()
print(columns[2]) #check für index 0,1,2 ist per hand passiert
x = columns[2]
final[x] = pd.to_numeric(final[x], downcast='integer')
root[x] = pd.to_numeric(root[x], downcast='integer')
print(root[x].equals(final[x]))  # Returns True)
print(type(columns))

#TEST 
# haben fixture_id bis timestamp gleiche Reihenfolge
# wenn ja sind in richtiger reiheinfolge konkateniert

timestamp
True
<class 'pandas.core.indexes.base.Index'>


In [136]:
for key in root.keys():
    print(key,":", all_norm[key].equals(final[key]))

#TEST
#expected output alles außer die ersten drei true
# haben alle werte außer die ersten drei gleiche Reihenfolge
# wenn ja sind in richtiger reiheinfolge konkateniert


fixture_id : False
player_id : False
timestamp : False
assists : True
blocks : True
commited_fouls : True
commited_penalty : True
conceded : True
drawn_fouls : True
dribble_attempts : True
dribble_past : True
dribble_success : True
interceptions : True
key_passes : True
minutes : True
missed_penalty : True
number : True
offside : True
on_goal_shots : True
pass_accuracy : True
rating : True
red : True
saved_penalty : True
saves : True
scored_penalty : True
total_duels : True
total_goals : True
total_passes : True
total_shots : True
total_tackels : True
won_duels : True
won_penalty : True
yellow : True


In [62]:
engine = create_engine(r'XXXX-XXXX', echo=False)
df.to_sql('avgPlayerStatistics', con=engine, if_exists='append') #if_exists='append'
test = pd.read_sql_query("SELECT  * FROM avgPlayerStatistics;", con)
display(test)

,index,assists,blocks,commited_fouls,commited_penalty,conceded,drawn_fouls,dribble_attempts,dribble_past,dribble_success,fixture_id,interceptions,key_passes,minutes,missed_penalty,number,offside,on_goal_shots,pass_accuracy,player_id,rating,red,saved_penalty,saves,scored_penalty,timestamp,total_duels,total_goals,total_passes,total_shots,total_tackels,won_duels,won_penalty,yellow
count,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000,322762.000
mean,0.480,0.035,0.025,0.000,0.006,0.020,0.092,0.076,0.096,0.071,0.247,0.122,0.095,0.717,0.003,0.175,0.000,0.063,0.608,0.076,0.462,0.007,0.002,0.022,0.010,0.504,0.197,0.034,0.201,0.087,0.070,0.164,0.007,0.076
std,0.289,0.071,0.054,0.000,0.026,0.072,0.077,0.084,0.086,0.081,0.289,0.112,0.099,0.182,0.024,0.161,0.000,0.086,0.193,0.080,0.323,0.042,0.024,0.075,0.053,0.300,0.132,0.066,0.112,0.092,0.095,0.113,0.037,0.095
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,0.231,0.000,0.000,0.000,0.000,0.000,0.040,0.013,0.029,0.000,0.024,0.024,0.028,0.640,0.000,0.071,0.000,0.000,0.556,0.006,0.000,0.000,0.000,0.000,0.000,0.248,0.106,0.000,0.117,0.020,0.000,0.073,0.000,0.000
50%,0.461,0.000,0.000,0.000,0.000,0.000,0.080,0.053,0.086,0.043,0.222,0.094,0.056,0.794,0.000,0.141,0.000,0.040,0.673,0.067,0.668,0.000,0.000,0.000,0.000,0.518,0.206,0.000,0.183,0.060,0.022,0.164,0.000,0.100
75%,0.730,0.100,0.040,0.000,0.000,0.000,0.140,0.107,0.143,0.106,0.296,0.188,0.139,0.857,0.000,0.232,0.000,0.080,0.738,0.118,0.698,0.000,0.000,0.000,0.000,0.762,0.283,0.067,0.265,0.120,0.111,0.236,0.000,0.100
max,1.000,1.000,1.000,0.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
